In [ ]:
# !pip3 install sklearn
# !pip3 install pandas
# !pip3 install openpyxl

## Import packages

In [ ]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
import datetime

## Import document as data frame

In [39]:
df = pd.read_excel('data/fluxo_escolar_filtrado.xlsx')

## Drop unused columns

In [41]:
drop_columns = ['NM_DIRETORIA', 'CD_REDE_ENSINO', 'CD_ESCOLA', 'APR_1', 'REP_1', 'ABA_1', 'APR_2', 'REP_2', 'ABA_2', 'Somatoria']
df_cleaned = df.drop(drop_columns, axis='columns')

## Renaming columns

In [42]:
columns_renamed = {
    'Ano':'ano', 
    'NM_MUNICIPIO':'municipio', 
    'NM_COMPLETO_ESCOLA':'escola', 
    'APR_3':'aprovado', 
    'REP_3':'reprovado', 
    'ABA_3':'abandono'
}
df_cleaned = df_cleaned.rename(columns=columns_renamed)

## Drop rows that contains zeros for 'aprovado' and 'reprovado' columns

In [43]:
df_cleaned = df_cleaned.loc[(df_cleaned['aprovado'] > 0) | (df_cleaned['reprovado'] > 0)]
df_cleaned.head()

,ano,municipio,escola,aprovado,reprovado,abandono
0,2015,FERRAZ DE VASCONCELOS,JOSE EDUARDO VIEIRA RADUAN DOUTOR,96.77,0.99,2.24
1,2015,FERRAZ DE VASCONCELOS,CONJUNTO HABITACIONAL PARQUE DOURADO II,75.62,20.66,3.72
3,2015,SUZANO,ROBERTO BIANCHI,97.14,1.71,1.15
5,2015,SUZANO,CARLOS MOLTENI PROFESSOR,90.33,9.67,0.00
6,2015,SUZANO,GIOVANNI BATTISTA RAFFO PROFESSOR DOUTOR,98.66,1.01,0.33


In [47]:
df_cleaned2 = df_cleaned.loc[(df_cleaned['aprovado'] + df_cleaned['reprovado'] + df_cleaned['abandono'] == 100)]
print(len(df_cleaned['municipio'].tolist()))
df_cleaned2.to_excel('output/fluxo_escolar_filtered_correct_percenteges.xlsx', encoding="utf-8")

26578


## Drop rows that contains NaN from data frame (cleaned)

In [34]:
df_cleaned = df_cleaned.dropna()

In [ ]:
#df_cleaned.head(20)

Exportando o resultado de hoje 07-11 23:31

In [37]:
df_cleaned.to_excel('fluxo_escolar_parcialmente_tratado.xlsx', encoding="utf-8")

DAQUI PARA BAIXO EU NÃO GARANTO NADA kkkkkk

---


1.   Garantir municipios unificados
2.   Grantir que não há caracteres "quebrados"




In [ ]:
# df_cleaned = df_cleaned[df_cleaned != "-"]
# df_cleaned = df_cleaned[df_cleaned != datetime.time(0, 0)]
# df_cleaned = df_cleaned.dropna()
df_cleaned.head(20)

In [ ]:
df_cleaned['ABA_3'] = df_cleaned['ABA_3'].astype(str)
df_cleaned = df_cleaned[df_cleaned["ABA_3"].str.contains("-") == False]

df_cleaned.head(20)

In [ ]:
df_cleaned['APR_3'] = df_cleaned['APR_3'].astype(float)
df_cleaned['REP_3'] = df_cleaned['REP_3'].astype(float)
df_cleaned['ABA_3'] = df_cleaned['ABA_3'].astype(float)

df_cleaned = df_cleaned.reset_index()
df_cleaned.head(20)

In [ ]:
normalized_list = preprocessing.normalize([df_cleaned['aprovado'],df_cleaned['reprovado'],df_cleaned['abandono']])

df_cleaned['aprovado_normalized'] = normalized_list[0]
df_cleaned['reprovado_normalized'] = normalized_list[1]
df_cleaned['abandono_normalized'] = normalized_list[2]

df_cleaned.head(20)